## 문제 1

### 1-1. 분류 평가지표 중 '정밀도'와 '재현율'에 대한 개념과 이 둘은 어떤 관계인지 설명해주세요.

### 1-2.  '정밀도'와 '재현율'의 관계로 인하여 다른 어떤 평가지표를 쓰는 것이 좋은지 이유와 함께 설명해주세요.

***

## 문제 2

### 정밀도와 재현율이 각각 더 중요한 사례를 들어주세요.
- 정밀도가 중요한 사례와 재현율이 중요한 사례 총 2가지를 적어주셔야 합니다.
- 되도록이면 PPT 이외의 사례를 들어주세요!

***

## 문제3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import Binarizer

In [3]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

### [ 종양종류 예측해보기 ]
- load_breast_cancer은 위스콘신 유방암 데이터로 악성종양(malignant)과 양성종양(benign)으로 구분되는 데이터세트 입니다.
- 타깃 레이블 값의 종류는 악성이 '0'으로 양성이 '1'로 되어 있습니다.

#### 기본 제공 코드

In [5]:
# 데이텃셋 로드 및 X,y 지정
dataset = load_breast_cancer()
X_features=dataset.data
y_label=dataset.target

In [6]:
# `train_test_split`으로 train, test 데이터 분리

X_train,X_test,y_train,y_test = train_test_split(X_features, y_label, test_size=0.3, random_state=11)
print(X_train.shape, X_test.shape)

(398, 30) (171, 30)


In [7]:
# RandomForest 객체 생성 후 예측 수행

from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state = 0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
pred_proba = rf_clf.predict_proba(X_test)[:,1]

### 3-1. 정확도, 정밀도, 재현율, 오차행렬을 산출하세요.
- get_clf_eval 함수를 활용하세요.

### 3-2. 임계값이 0.3, 0.5, 0.7, 0.9일때 오차행렬, 정확도, 정밀도, 재현율을 구하세요.

In [8]:
thresholds = [ 0.3, 0.5, 0.7, 0.9]

### 3-3. 위의 임곗값들 중 어떤 것이 가장 적절할까요? 답과 그 이유를 적어주세요.

***

## 문제 4

### 4-1. F1 스코어의 개념과 어떨 때 상대적으로 높은 값를 가지는지 적어주세요.

### 4-2. F1 스코어를 구하세요.

### 4-3. `get_clf_eval()` 함수에 F1 스코어를 구하는 코드를 추가하세요.

### 4-4. <U>F1 스코어 >= 0.98</U>이 되도록 값을 출력하세요.  (다른 평가 지표의 값은 무시)
  
`Hint : 임계값 조정`

***

## 문제 5

### 5-1. ROC-AUC가 무엇인지 설명하세요. 

In [14]:
# ROC 곡선 : 
# TPR(True Positive Rate) : 
# TNR(True Negative Rate) : 
# AUC(Area Under Curve) : 

### 5-2. ROC-AUC 값을 구하세요. 